# Experimento con LanceDB (prompttools)

[Prompt Tools](https://github.com/hegelai/prompttools/tree/main/examples/notebooks)

## Instalación de paquetes de python

In [1]:
!pip install --quiet --force-reinstall prompttools lancedb sentence_transformers openai pandas

In [ ]:
!pip list | grep lance

## El propio experimento carga lancedb

[LanceDBExperiment](https://github.com/hegelai/prompttools/blob/main/prompttools/experiment/experiments/lancedb_experiment.py)

In [ ]:
from prompttools.experiment import LanceDBExperiment

## Comparando distintos embeddings con LanceDB

Un caso de uso común es comparar dos modelos de embeddings diferentes y cómo puede afectar a la recuperación de documentos. Aquí podemos definir qué modelos de embeddings queremos probar.

Nota: Si previamente no ha descargado estos modelos de embeddings. Esto puede iniciar las descargas.

In [ ]:
from sentence_transformers import SentenceTransformer
import openai
import os

# Comprobamos si la variable de entorno de OpenAI OPENAI_API_KEY 
if "OPENAI_API_KEY" not in os.environ:
    # O la podemos indicar directamente, si no tenemos la variable de entorno visible.
    openai.api_key = "..."


DEFAULT = SentenceTransformer("paraphrase-MiniLM-L3-v2")
MIMNILM_L6 = SentenceTransformer("all-MiniLM-L6-v2")


def default_embed_func(batch):
    return [DEFAULT.encode(sentence) for sentence in batch]


def minilm_l6_embed_func(batch):
    return [MIMNILM_L6.encode(sentence) for sentence in batch]


def openai_ada2_embed_func(batch):
    rs = openai.Embedding.create(input=batch, engine="text-embedding-ada-002")
    return [record["embedding"] for record in rs["data"]]


emb_fns = {"minilm_l6": minilm_l6_embed_func, "default": default_embed_func}
# Prueba con los embeddings de openAI
# emb_fns = {"openai-ada-002": openai_ada2_embed_func, "minilm_l6": minilm_l6_embed_func,  "default": default_embed_func }


Durante el experimento, para cada modelo de embeddings se creará temporalmente una nueva tabla en LanceDB. En ella se añadirán los documentos. A continuación, realizaremos una consulta a partir de ella y examinaremos los resultados.

In [ ]:
import pandas as pd

use_existing_table = False  # Esto sirve para indicar que crearemos una tabla al vuelo.


Documentos que se añadirán a la base de datos. LanceDB también acepta otros formatos de conjuntos de datos como **pydict, pyarrow, Pydantic, o incluso ficheros parquet** (que se emplean mucho en Big Data).
[Más información](https://lancedb.github.io/lancedb/guides/tables/)


In [ ]:

data = pd.DataFrame(
    {
        "text": ["This is a document", "This is another document", "This is the document."],
        "metadatas": [{"source": "my_source"}, {"source": "my_source"}, {"source": "my_source"}],
        "ids": ["id1", "id2", "id3"],
    }
)

query_args = {"text": ["This is a query document", "This is a another query document"], "metric": ["cosine", "l2"]}


# Configuramos el experimento
experiment = LanceDBExperiment(
    data=data,
    embedding_fns=emb_fns,
    query_args=query_args,
)

# [Opcional] Argumentos de consulta avanzados
# Nuestras consultas de prueba, junto con argumentos de consulta opcionales. La consulta LanceDB acepta algunos argumentos para personalizar la búsqueda:
# métricas: «l2», «coseno», o «punto» (coseno por defecto)
# filtro: Cláusula SQL where para filtrar los resultados de la búsqueda vectorial antes de aplicar el límite. (Ninguna por defecto)
# limit: número de resultados a devolver (3 por defecto)
"""
query_args_adv = {
                "text": ["This is a query document", "This is a another query document"], 
                "metric": ["cosine", "l2", "dot"],
                "filter": ["text IS NOT NULL" , "text LIKE '%document.%'"]
                }
experiment = LanceDBExperiment(
    data=data,
    embedding_fns=emb_fns,
    query_args=query_args_adv,
 
)
"""

Ejecutamos el experimento con esta llamada.

In [ ]:
experiment.run()

Visualizamos los resultados

In [ ]:
experiment.visualize()

## Evaluar los resultados

Para evaluar los resultados, definiremos una función de evaluación. A veces, se conoce el orden del documento más relevante dada una consulta, y se puede calcular la correlación entre el ranking esperado y el ranking real.

In [ ]:
import scipy.stats as stats

# Aquí definimos cual sería el ranking ideal, para poder contrastarlo con el ejecutado en el experimento
EXPECTED_RANKING = {
    "This is a query document": ["id1", "id3", "id2"],
    "This is a another query document": ["id2", "id3", "id1"],
}


def measure_correlation(row: "pandas.core.series.Series", ranking_column_name: str = "top doc ids") -> float:
    r"""
    A simple test that compares the expected ranking for a given query with the actual ranking produced
    by the embedding function being tested.
    """
    input_query = row["text"]
    correlation, _ = stats.spearmanr(row[ranking_column_name], EXPECTED_RANKING[input_query])
    return correlation

Finalmente visualizamos los resultados.

In [ ]:
experiment.evaluate("ranking_correlation", measure_correlation)

In [ ]:
experiment.visualize()

# Próximos pasos para practicar:

Intenta adaptar este código para que los embeddings los importe nativamente desde lancedb. 
Referencias:
- [Embeddings en LanceDB](https://lancedb.github.io/lancedb/embeddings/default_embedding_functions/)
- [Más ejemplos de LanceDB](https://github.com/lancedb/vectordb-recipes)